# Assignment 1

## Exercise 1.1

rough sketch for ex 1.2:

1. define the „positive“ subspace P in the RGB cube
2. iterate over all pixels in I and check if in P or ~P
3. write result to new image
4. play around with size and shape of P and display binary image (**RESULT**)



In [6]:
%matplotlib inline 
from skimage import io, data, color
import numpy as np

image = io.imread('testbilder/1_klein.jpg')
schwellwert = [104,22,43]
bereich=30

for line in image:
    for pixel in line:
        isRCase=pixel[0] >(schwellwert[0]+bereich) or pixel[0]<(schwellwert[0]-bereich)
        isGCase=pixel[1] >(schwellwert[1]+bereich) or pixel[1]<(schwellwert[1]-bereich)
        isBCase=pixel[2] >(schwellwert[2]+bereich) or pixel[2]<(schwellwert[2]-bereich)
        if isRCase and isGCase and isBCase:
            pixel=[255,255,255]
        else:
            pixel=[0,0,0]



hallo
[[254 242 226]
 [254 241 224]
 [255 241 222]
 ...
 [254 245 236]
 [254 244 234]
 [252 242 232]]


## Exercise 1.2

* starting from the binary color detection image
* erase noise with an erosion operation
* dilate once to get original size of object
* find connected components with one-pass algorithm
* extract bounding box on the fly
* draw bounding box on original image (**RESULT**)

## Exercise 1.3

* use your color detection and connected components algorithm
* implement simplest tracking algorithm
* draw history of all previous points on frame (**RESULT**)